In [ ]:
# Install the pytorch and ultralytics 
%pip install -U ultralytics
%pip install torch torchvision torchaudio
%pip install pandas

In [2]:
import ultralytics 
#import cv2
import pandas as pd
from distutils.dir_util import copy_tree
import os
# Check  GPU  and pytorch works
ultralytics.checks()

# It should list GPU device ex:  
# Ultralytics YOLOv8.1.0 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
# Setup complete ✅ (16 CPUs, 31.1 GB RAM, 358.8/915.3 GB disk)

Ultralytics 8.3.1 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Setup complete ✅ (16 CPUs, 31.1 GB RAM, 501.5/915.3 GB disk)


# XML File Placement Guide

To ensure proper functionality, please place the required XML file in the designated folder following these steps:

1. **Locate the XML File**  
   Ensure that you have the XML file you need to upload. This file contains essential data for the application to operate correctly.

2. **Navigate to the Designated Folder**  
   Open your file explorer and navigate to the following path:


In [10]:

# Ge the list of files in the directory 
fr = os.listdir('./Xml_to_Yolotxt/xml')


['T16L600_W1_GC1_R2_P74_g55_3.xml']


In [14]:
#coding:utf-8


"""
<object>
    <type>robndbox</type>
    <name>wenben</name>
    <pose>Unspecified</pose>
    <truncated>0</truncated>
    <difficult>0</difficult>
    <robndbox>
        <cx>860.5666</cx>
        <cy>734.5734</cy>
        <w>644.8657</w>
        <h>52.3775</h>
        <angle>3.031593</angle>
    </robndbox>
</object>
cx and cy represent the coordinates of the center point of the bndbox (the upper left corner of the coordinate system is the origin, 
the right is the positive x direction, and the downward is the positive y direction); h and w are the height and width of the character block; 
angle is the rotation angle information. It should be noted here that the rotation angle value obtained by the roLabelImg annotation is as follows: 
first, draw a horizontal bndbox, angle = 0 at this time. If you rotate clockwise, the angle value obtained is a positive value of a radian unit. 
According to this idea, if you rotate counterclockwise, the angle value obtained should be a negative value of a radian unit, but this is not the case. 
For example, in the example above, the actual angle should be rotated a little counterclockwise, but the angle value obtained is 3.081593, 
which is also a positive value. Assuming that the angle is negative when rotating counterclockwise, then its angle theta should be: theta = angle-pi,
"""

import os
import numpy as np

try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET
import sys

import math


def rotate(angle, x, y):
    """
    
    :param angle:   radian
    :param x:       x
    :param y:       y
    :return:
    """
    rotatex = math.cos(angle) * x - math.sin(angle) * y
    rotatey = math.cos(angle) * y + math.sin(angle) * x
    return rotatex, rotatey

def xy_rorate(theta, x, y, centerx, centery):
    """
    
    :param theta:
    :param x:
    :param y:
    :param centerx:
    :param centery:
    :return:
    """
    r_x, r_y = rotate(theta, x - centerx, y - centery)
    return centerx+r_x, centery+r_y

def rec_rotate(x, y, width, height, theta):
    """
    
    :param x:
    :param y:
    :param width:
    :param height:
    :param theta:
    :return:
    """
    centerx = x + width / 2
    centery = y + height / 2

    x1, y1 = xy_rorate(theta, x, y, centerx, centery)
    x2, y2 = xy_rorate(theta, x+width, y, centerx, centery)
    x3, y3 = xy_rorate(theta, x, y+height, centerx, centery)
    x4, y4 = xy_rorate(theta, x+width, y+height, centerx, centery)

    return x1, y1, x2, y2, x4, y4,x3, y3


def test(x,y,cx,cy,theta):
    x1_test = cx+(x-cx)*math.cos(theta)-(y-cy)*math.sin(theta)
    y1_test = cy+(y-cy)*math.cos(theta)+(x-cx)*math.sin(theta)
    print('x1_testx1_testx1_testx1_testx1_test',x1_test)
    print('y1_testy1_testy1_testy1_testy1_test',y1_test)


PATH = './Xml_to_Yolotxt/'



class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                "blurry.complete": 2,
                "blurry.incomplete" :3 ,
                "hair":4
                } # 


xml_path = os.path.join(PATH +'/xml/')
#print(xml_path)
txt_path = os.path.join(PATH + '/Yolotxt/')
# print(xml_path)

for line in fr:
    if line:
        tree = ET.parse(os.path.join(xml_path,line.strip()))     # open xml file
        root = tree.getroot()         # get root node
        filename = root.find('filename').text
        file_object = open(os.path.join(txt_path,filename + ".txt"), 'w') # write txt file
        # file_object_log = open(filename + ".log", 'w') #write log file
        flag = False

    for size in root.findall('size'): #
        width = int(size.find('width').text)   #
        height = int(size.find('height').text)   #

    for object in root.findall('object'): #
        name = object.find('name').text
        robndbox = object.find('robndbox')
        print(filename)
        cx = float(robndbox.find('cx').text)
        cy = float(robndbox.find('cy').text)
        w = float(robndbox.find('w').text)
        h = float(robndbox.find('h').text)
        angle = float(robndbox.find('angle').text)

        x = cx - w/2
        y = cy - h/2
        if angle<1.57:
            theta = round(angle, 6)
        else:
            theta = round(angle - np.pi, 6)
        x1, y1, x2, y2, x4, y4,x3, y3 = rec_rotate(x, y, w, h, theta)
        x1,y1,x2,y2,x4,y4,x3,y3 = int(x1) / width,int(y1) / height,int(x2) / width ,int(y2) / height ,int(x4) /width,int(y4) / height,int(x3) / width,int(y3) /height
        #print(filename, x1, y1, x2, y2, x4, y4,x3, y3)

        test(x,y,cx,cy,theta)

        file_object.write(str(class_name_to_id_mapping[name]) + ' '+ str(x1)+' '+str(y1)+' '+str(x2)+' '+str(y2)+' '+str(x4)+' '+str(y4)+' '+str(x3)+' '+str(y3))
        file_object.write('\n')
    file_object.close()

./Xml_to_Yolotxt//xml/
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 265.0
y1_testy1_testy1_testy1_testy1_test 42.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 615.0
y1_testy1_testy1_testy1_testy1_test 48.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 1019.0
y1_testy1_testy1_testy1_testy1_test 74.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 1435.0
y1_testy1_testy1_testy1_testy1_test 70.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 1995.0
y1_testy1_testy1_testy1_testy1_test 60.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 2398.0
y1_testy1_testy1_testy1_testy1_test 75.99995000000001
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 199.30581191452197
y1_testy1_testy1_testy1_testy1_test 515.1068188234173
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1_testx1_testx1_test 1001.0
y1_testy1_testy1_testy1_testy1_test 500.0
T16L600_W1_GC1_R2_P74_g55_3
x1_testx1_testx1

# Valid Pipeline

### The default path for valid images and labels is specified in validBerlinHU.yaml

In [19]:
from ultralytics import YOLO

model = YOLO('./T22without_pipeline.pt')

metric = model.val(data = 'validBerlinHU.yaml')




Ultralytics 8.3.1 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
YOLOv8m-obb summary (fused): 255 layers, 23,766,034 parameters, 0 gradients, 69.6 GFLOPs


val: Scanning /home/ray/BerlinHUYoloStomata/dataset/labels/val... 1 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1/1 [00:00<00:00, 351.84it/s]

val: New cache created: /home/ray/BerlinHUYoloStomata/dataset/labels/val.cache



/home/ray/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.67it/s]


                   all          1         23      0.988      0.997      0.995      0.911
       blurry.complete          1         21          1      0.995      0.995      0.926
     blurry.incomplete          1          2      0.975          1      0.995      0.895
Speed: 0.5ms preprocess, 93.6ms inference, 0.0ms loss, 92.4ms postprocess per image
Results saved to /home/ray/BerlinHUYoloStomata/runs/obb/val5
